## Function calling

This new feature proves beneficial in a wide array of situations. It can assist in designing chatbots capable of interacting with various APIs, facilitate task automation, and enable extraction of organized information from inputs expressed in natural language.

In [1]:
# !pip install --upgrade langchain
# !pip install python-dotenv
# !pip install openai

Ensure you got the AT LEAST Version 0.0.200

In [2]:
import pkg_resources


def print_version(package_name):
    try:
        version = pkg_resources.get_distribution(package_name).version
        print(f"The version of the {package_name} library is {version}.")
    except pkg_resources.DistributionNotFound:
        print(f"The {package_name} library is not installed.")


print_version("langchain")

C:\Users\riyag\AppData\Local\Temp\ipykernel_3804\2081299685.py:1: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


The version of the langchain library is 0.1.8.


In [3]:
from dotenv import load_dotenv
import os
import openai
import json
import warnings
warnings.filterwarnings("ignore")
from secret_key import openapi_key

os.environ['OPENAI_API_KEY'] = openapi_key



In [12]:
openai.api_key=openapi_key

First, you need to define a function. This one mimics a database query with a "fake" price.

In [13]:
def get_pizza_info(pizza_name: str):
    pizza_info = {
        "name": pizza_name,
        "price": "10.99",
    }
    return json.dumps(pizza_info)

In [14]:
functions = [
    {
        "name": "get_pizza_info",
        "description": "Get name and price of a pizza of the restaurant",
        "parameters": {
            "type": "object",
            "properties": {
                "pizza_name": {
                    "type": "string",
                    "description": "The name of the pizza, e.g. Salami",
                },
            },
            "required": ["pizza_name"],
        },
    }
]

In [15]:
def chat(query):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[{"role": "user", "content": query}],
        functions=functions,
    )
    message = response["choices"][0]["message"]
    return message

In [16]:
chat("What is the capital of france?")

<OpenAIObject at 0x1c8274aa110> JSON: {
  "role": "assistant",
  "content": "The capital of France is Paris."
}

In [17]:
query = "How much does pizza salami cost?"
message = chat(query)
message

<OpenAIObject at 0x1c8274aaca0> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_pizza_info",
    "arguments": "{\n  \"pizza_name\": \"Salami\"\n}"
  }
}

In [18]:
if message.get("function_call"):
    function_name = message["function_call"]["name"]
    pizza_name = json.loads(message["function_call"]["arguments"]).get("pizza_name")
    print(pizza_name)
    function_response = get_pizza_info(
        pizza_name=pizza_name
    )

    second_response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[
            {"role": "user", "content": query},
            message,
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            },
        ],
    )

second_response

Salami


<OpenAIObject chat.completion id=chatcmpl-95pV1rndWoWPYnxCDaStDDGNx1W26 at 0x1c8274b6980> JSON: {
  "id": "chatcmpl-95pV1rndWoWPYnxCDaStDDGNx1W26",
  "object": "chat.completion",
  "created": 1711176631,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The cost of a pizza salami is $10.99."
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 59,
    "completion_tokens": 13,
    "total_tokens": 72
  },
  "system_fingerprint": null
}

### LangChain 

LangChain allows you to use functions too, but currently (15.06.2023) it is kind of hacky to do it. 


In [19]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, ChatMessage

In [20]:
llm = ChatOpenAI(model="gpt-3.5-turbo-0613")
message = llm.predict_messages(
    [HumanMessage(content="What is the capital of france?")], functions=functions
)
message

d:\IIA\Tweet data and llm\env\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
d:\IIA\Tweet data and llm\env\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `predict_messages` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
Retrying langchain_community.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-3.5-turbo-0613 in organization org-GBzolOmzUHWBjRNhBNkzVVS1 on requests per min (RPM): 

AIMessage(content='The capital of France is Paris.')

In [22]:
query = "How much does Pizza Salami cost in the restaurant?"
message_pizza = llm.predict_messages([HumanMessage(content=query)], functions=functions)
message_pizza

AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_pizza_info', 'arguments': '{\n"pizza_name": "Salami"\n}'}})

In [23]:
message.additional_kwargs

{}

In [24]:
message_pizza.additional_kwargs

{'function_call': {'name': 'get_pizza_info',
  'arguments': '{\n"pizza_name": "Salami"\n}'}}

In [25]:
import json

pizza_name = json.loads(message_pizza.additional_kwargs["function_call"]["arguments"]).get("pizza_name")
pizza_name

'Salami'

In [26]:
pizza_api_response = get_pizza_info(pizza_name=pizza_name)
pizza_api_response

'{"name": "Salami", "price": "10.99"}'

In [27]:


second_response = llm.predict_messages(
    [
        HumanMessage(content=query),
        AIMessage(content=str(message_pizza.additional_kwargs)),
        ChatMessage(
            role="function",
            additional_kwargs={
                "name": message_pizza.additional_kwargs["function_call"]["name"]
            },
            content=pizza_api_response
        ),
    ],
    functions=functions,
)
second_response

AIMessage(content='The Pizza Salami costs $10.99 in the restaurant.')

### Use Tools

You can convert Tools to functions, both the Tools provided by langchain and also your own tools. The Tool you see is there to show you the interface of a tool, it actually does nothing really do anything useful ;-) 

In [28]:
from typing import Optional
from langchain.tools import BaseTool
from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)


class StupidJokeTool(BaseTool):
    name = "StupidJokeTool"
    description = "Tool to explain jokes about chickens"

    def _run(
        self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        return "It is funny, because AI..."

    async def _arun(
        self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("joke tool does not support async")

In [29]:
from langchain.tools import format_tool_to_openai_function, MoveFileTool


tools = [StupidJokeTool(), MoveFileTool()]
functions = [format_tool_to_openai_function(t) for t in tools]
functions

d:\IIA\Tweet data and llm\env\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `format_tool_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 0.2.0. Use langchain_core.utils.function_calling.convert_to_openai_function() instead.
  warn_deprecated(


[{'name': 'StupidJokeTool',
  'description': 'Tool to explain jokes about chickens',
  'parameters': {'properties': {'__arg1': {'title': '__arg1',
     'type': 'string'}},
   'required': ['__arg1'],
   'type': 'object'}},
 {'name': 'move_file',
  'description': 'Move or rename a file from one location to another',
  'parameters': {'type': 'object',
   'properties': {'source_path': {'description': 'Path of the file to move',
     'type': 'string'},
    'destination_path': {'description': 'New path for the moved file',
     'type': 'string'}},
   'required': ['source_path', 'destination_path']}}]

In [31]:
query = "Why does the chicken cross the road? To get to the other side"
output = llm.predict_messages([HumanMessage(content=query)], functions=functions)
output

AIMessage(content='', additional_kwargs={'function_call': {'name': 'StupidJokeTool', 'arguments': '{\n  "__arg1": "To get to the other side"\n}'}})

In [32]:
question = json.loads(output.additional_kwargs["function_call"]["arguments"]).get("__arg1")
tool_response = tools[0].run(question)
tool_response

'It is funny, because AI...'

In [33]:
second_response = llm.predict_messages(
    [
        HumanMessage(content=query),
        AIMessage(content=str(output.additional_kwargs)),
        ChatMessage(
            role="function",
            additional_kwargs={
                "name": output.additional_kwargs["function_call"]["name"]
            },
            content="""
                {tool_response}
            """,
        ),
    ],
    functions=functions,
)
second_response

AIMessage(content='', additional_kwargs={'function_call': {'name': 'StupidJokeTool', 'arguments': '{\n  "__arg1": "To get to the other side"\n}'}})

### OpenAI Functions Agent

In [34]:
from langchain import LLMMathChain
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI


llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)
tools = [
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math"
    ),
]

In [35]:
agent = initialize_agent(tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True)

d:\IIA\Tweet data and llm\env\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [36]:
agent.run("What is the capital of france?")

d:\IIA\Tweet data and llm\env\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
The capital of France is Paris.

> Finished chain.


'The capital of France is Paris.'

In [42]:
agent.run("What is 100 devided by 25?")



> Entering new AgentExecutor chain...

Invoking: `Calculator` with `100 / 25`




> Entering new LLMMathChain chain...
100 / 25```text
100 / 25
```
...numexpr.evaluate("100 / 25")...

Answer: 4.0
> Finished chain.
Answer: 4.0100 divided by 25 is equal to 4.

> Finished chain.


'100 divided by 25 is equal to 4.'